In [22]:
import numpy as np

a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
c = np.dstack([a, b])  # 열방향으로 배열 결합

print(a)
print(b)
print("--------")
print(c)

[[1 2]
 [3 4]]
[[5 6]
 [7 8]]
--------
[[[1 5]
  [2 6]]

 [[3 7]
  [4 8]]]


In [23]:
%autosave 0
import numpy as np
import tensorflow as tf

# tensorflow 2.0에 내장된 Keras 사용
from tensorflow.keras.models import Sequential  # class
from tensorflow.keras.models import load_model  # model 사용 함수
from tensorflow.keras.layers import Dense       # 전결합층
from tensorflow.keras.optimizers import Adam    # 가중치, bias 최적화
from tensorflow.keras.callbacks import Callback

# tensorflow 1.x, Keras가 독립적으로 설치된 경우
# from keras.models import Sequential  # class
# from keras.models import load_model  # model 사용 함수
# from keras.layers import Dense       # class

print(tf.__version__)

Autosave disabled
2.0.0


In [24]:
# 데이터
x_train = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,15,16,17,18,19,20])


y_train = np.array([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

print(x_train)
print(y_train)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40]


In [31]:
from tensorflow.keras.callbacks import Callback

class GetWeights(Callback):
    # 가중치와 Bias를 저장할 Dictionary 선언 
    def __init__(self):
        super(GetWeights, self).__init__()
        self.weight_dict = {}

    # 각각의 학습이 끝난 후 실행       
    def on_epoch_end(self, epoch, logs=None):
        
        # Layer 수만큼 순환 하면서 weights, biases 수집
        for layer_i in range(len(self.model.layers)):
            w = self.model.layers[layer_i].get_weights()[0] # 가중치
            b = self.model.layers[layer_i].get_weights()[1] # 편향
            print('Layer %s has weights of shape %s and biases of shape %s' %(layer_i, np.shape(w), np.shape(b)))
            print('Epoch:', epoch)

            # 첫번째 학습(0)이라면 np.dstack 함수를 사용하지 말고 dictionary에 weights, biases 저장
            if epoch == 0:
                self.weight_dict['w_'+str(layer_i+1)] = w
                self.weight_dict['b_'+str(layer_i+1)] = b
                
            else: # 두번째 학습부터 이전에 만든 가중치 배열에 새 가중치를 추가
                self.weight_dict['w_'+str(layer_i+1)] = np.dstack((self.weight_dict['w_'+str(layer_i+1)], w))
                self.weight_dict['b_'+str(layer_i+1)] = np.dstack((self.weight_dict['b_'+str(layer_i+1)], b))

### 1 epoch, , 전체 데이터 1회 학습

In [32]:
model = Sequential()
model.add(Dense(3, input_dim=1, activation='linear'))
model.add(Dense(2, input_dim=1, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=0.001), loss='mse')

gw = GetWeights()
hist = model.fit(x_train, y_train, validation_split=0.2, shuffle=True,
                 epochs=1, batch_size=1, callbacks=[gw])

Train on 16 samples, validate on 4 samples
 1/16 [>.............................] - ETA: 5s - loss: 654.5071Layer 0 has weights of shape (1, 3) and biases of shape (3,)
Epoch: 0
Layer 1 has weights of shape (3, 2) and biases of shape (2,)
Epoch: 0
Layer 2 has weights of shape (2, 1) and biases of shape (1,)
Epoch: 0
16/16 [==============================] - 0s 29ms/sample - loss: 1207.9048 - val_loss: 4344.8010


In [33]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 3)                 6         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 3         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


In [34]:
for key in gw.weight_dict:
    print(str(key) + ' shape: %s' %str(np.shape(gw.weight_dict[key])))  # 면, 행, 열
    print(gw.weight_dict[key])
    print('---------------------------------------')

w_1 shape: (1, 3)
[[-0.3170681 -1.1735408  1.2059734]]
---------------------------------------
b_1 shape: (3,)
[-0.01490724  0.01488336 -0.0148894 ]
---------------------------------------
w_2 shape: (3, 2)
[[ 0.8647011   0.04561592]
 [-0.8217976   0.28319812]
 [-0.34066963 -0.8109783 ]]
---------------------------------------
b_2 shape: (2,)
[-0.01488858  0.01489512]
---------------------------------------
w_3 shape: (2, 1)
[[-0.36015198]
 [ 1.1013905 ]]
---------------------------------------
b_3 shape: (1,)
[0.01489829]
---------------------------------------


### 2 epoch, 전체 데이터 2회 학습

In [35]:
model = Sequential()
model.add(Dense(3, input_dim=1, activation='linear'))
model.add(Dense(2, input_dim=1, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=0.001), loss='mse')

gw = GetWeights()
hist = model.fit(x_train, y_train, validation_split=0.2, shuffle=True,
                 epochs=2, batch_size=1, callbacks=[gw])

Train on 16 samples, validate on 4 samples
Epoch 1/2
 1/16 [>.............................] - ETA: 4s - loss: 149.0112Layer 0 has weights of shape (1, 3) and biases of shape (3,)
Epoch: 0
Layer 1 has weights of shape (3, 2) and biases of shape (2,)
Epoch: 0
Layer 2 has weights of shape (2, 1) and biases of shape (1,)
Epoch: 0
16/16 [==============================] - 0s 23ms/sample - loss: 844.5253 - val_loss: 3023.2486
Epoch 2/2
 1/16 [>.............................] - ETA: 0s - loss: 1265.5430Layer 0 has weights of shape (1, 3) and biases of shape (3,)
Epoch: 1
Layer 1 has weights of shape (3, 2) and biases of shape (2,)
Epoch: 1
Layer 2 has weights of shape (2, 1) and biases of shape (1,)
Epoch: 1
16/16 [==============================] - 0s 3ms/sample - loss: 800.6659 - val_loss: 2863.5303


In [29]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 3)                 6         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 3         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


In [30]:
for key in gw.weight_dict:
    print(str(key) + ' shape: %s' %str(np.shape(gw.weight_dict[key])))  # 면, 행, 열
    print(gw.weight_dict[key])
    print('---------------------------------------')

w_1 shape: (1, 3, 2)
[[[ 0.48732367  0.47760004]
  [-0.2644307  -0.27431837]
  [ 1.1001312   1.090366  ]]]
---------------------------------------
b_1 shape: (1, 3, 2)
[[[-0.01317578 -0.02461604]
  [-0.01321832 -0.02483793]
  [-0.01318648 -0.02467193]]]
---------------------------------------
w_2 shape: (3, 2, 2)
[[[ 0.14545678  0.15515743]
  [ 0.9897262   0.9800153 ]]

 [[-0.89801145 -0.90801907]
  [ 0.3860237   0.39604157]]

 [[-0.98956233 -0.97980255]
  [ 0.947953    0.9381828 ]]]
---------------------------------------
b_2 shape: (1, 2, 2)
[[[ 0.01319745  0.0247292 ]
  [-0.01320016 -0.02474324]]]
---------------------------------------
w_3 shape: (2, 1, 2)
[[[-0.85921186 -0.84958345]]

 [[ 1.0689827   1.0592709 ]]]
---------------------------------------
b_3 shape: (1, 1, 2)
[[[-0.01321137 -0.02480177]]]
---------------------------------------
